<a href="https://colab.research.google.com/github/j-phi/IPEDS/blob/master/IPEDS_Category_Filler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![IPEDS LOGO](https://drive.google.com/uc?export=view&id=1OgR-yDdjZToLc56BxF-oaA9zRK7iDKva)](https://nces.ed.gov/ipeds/datacenter/Data.aspx)

# IPEDS Compare Institutions Datacenter Category Decoder

IPEDS has great ["Compare Institutions"](https://nces.ed.gov/ipeds/datacenter/Data.aspx) feature that allows for large-scale downloading of data on multiple institutions. The challenge is that all of the columns are automatically encoded as numeric values by default. 

As an example, a University might be classifiable into two categories: 'Two Year' and 'Four Year'. In the dataset, instead of listing those values, the value will be encoded as '1' and '2'. You have the option of downloading a separate spreadsheet of value labels that explains the encoding for every column, but with most columns being encoded and many columns having a high number of possible options, it's too cumbersome to quickly analyze without an analytics platform like SAS or SPSS. 

As such, this script will automate the process of decoding the data. It accepts the data file and the value labels file, and then automatically re-converts the numerical representations into their original form.

Sample files are available here:
* [Data](https://drive.google.com/file/d/15dWlNWpp-Q7hEEUus6oHueDAsRZVqw3B/view?usp=sharing)
* [Value Labels](https://drive.google.com/file/d/1EuB6lfYhY6P_7U3eXu5i2BDziJMm4skB/view?usp=sharing)


## Imports and Settings

In [0]:
from google.colab import files
import sys
import pandas as pd
import numpy as np
from datetime import datetime as dt
from ipywidgets import widgets
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None

## Upload Files

To begin, upload **both your data and value label files** from your IPEDS download. To get the value label files, note that you have to select `"Yes"` to `"Do you want to include value labels?"` on the final data download step.

If this throws an error message, just re-run the cell until you have a `Browse...` button without the error message.

In [3]:
uploaded = files.upload()

for fn in uploaded.keys():
  print(f'[✓] Uploaded file "{fn}" with length {round(len(uploaded[fn])/1000000,2):,}MB')

Saving Data_9-2-2019---119.csv to Data_9-2-2019---119 (8).csv
Saving ValueLabels_9-2-2019---119.csv to ValueLabels_9-2-2019---119 (8).csv
[✓] Uploaded file "Data_9-2-2019---119.csv" with length 1.29MB
[✓] Uploaded file "ValueLabels_9-2-2019---119.csv" with length 0.1MB


Let's extract the relevant file names.

In [16]:
data_loc = ''
labels_loc = ''

for fn in uploaded.keys():
  if 'data' in fn.lower():
    data_loc = fn
  elif 'valuelabels' in fn.lower():
    labels_loc = fn

datavaluestr = f'[✓] Data and Value Label Files identified. \
Data file: {data_loc} | Value label file: {labels_loc}'

if data_loc != '' and labels_loc != '':
  print(datavaluestr)
else:
  print(f'It does not look like you uploaded your data and valuelabel files. Here is what we have: \n {datavaluestr}')

[✓] Data and Value Label Files identified. Data file: Data_9-2-2019---119.csv | Value label file: ValueLabels_9-2-2019---119.csv


## Decode the Files

Next, we'll ingest our data from .csv files into pandas dataframes. We'll call them `df` for the dataframe that houses our data and `labels` for the dataframe that houses our labels. 

In [0]:
df = pd.read_csv(data_loc)
labels = pd.read_csv(labels_loc)

In [36]:
df.head()

,UnitID,Institution Name,FIPS state code (HD2017),Institution (entity) name (HD2017),Institution name alias (HD2017),State abbreviation (HD2017),City location of institution (HD2017),ZIP code (HD2017),Fips County code (HD2017),Longitude location of institution (HD2017),Latitude location of institution (HD2017),Status of institution (HD2017),Sector of institution (HD2017),Level of institution (HD2017),Control of institution (HD2017),Historically Black College or University (HD2017),Degree of urbanization (Urban-centric locale) (HD2017),Carnegie Classification 2015: Undergraduate Instructional Program (HD2017),Carnegie Classification 2015: Graduate Instructional Program (HD2017),Carnegie Classification 2015: Enrollment Profile (HD2017),Carnegie Classification 2015: Undergraduate Profile (HD2017),Carnegie Classification 2015: Size and Setting (HD2017),Land Grant Institution (HD2017),Institution size category (HD2017),Institutional category (HD2017),Bureau of Economic Analysis (BEA) regions (HD2017),Does institution have a tenure system (FLAGS2017),Total price for in-state students living on campus 2017-18 (DRVIC2017),Total price for out-of-state students living on campus 2017-18 (DRVIC2017),Percent of full-time first-time undergraduates awarded Pell grants (SFA1617),Average amount of Pell grant aid awarded to full-time first-time undergraduates (SFA1617),Total enrollment (DRVEF2017),Unnamed: 32
0,177834,A T Still University of Health Sciences,29,A T Still University of Health Sciences,NaN,MO,Kirksville,63501,29001,-92.589183,40.193648,A,2,1,2,2,33,0,18,7,0,18,2,2,1,4,1,NaN,NaN,NaN,NaN,3723.0,NaN
1,180203,Aaniiih Nakoda College,30,Aaniiih Nakoda College,NaN,MT,Harlem,59526,30005,-108.757816,48.484196,A,4,2,1,2,43,15,0,1,1,1,1,1,4,7,2,NaN,NaN,85.0,4091.0,122.0,NaN
2,459523,ABC Beauty Academy,48,ABC Beauty Academy,NaN,TX,Richardson,75081,48113,-96.685333,32.931698,A,9,3,3,2,12,-2,-2,-2,-2,-2,2,1,6,6,2,NaN,NaN,0.0,NaN,NaN,NaN
3,485500,ABCO Technology,6,ABCO Technology,NaN,CA,Inglewood,90304-1551,6037,-118.369774,33.932121,A,9,3,3,2,21,-2,-2,-2,-2,-2,2,1,6,8,2,NaN,NaN,50.0,4479.0,50.0,NaN
4,461892,Abcott Institute,26,Abcott Institute,NaN,MI,Southfield,48075-5227,26125,-83.210617,42.446331,A,9,3,3,2,13,-2,-2,-2,-2,-2,2,1,6,3,2,NaN,NaN,100.0,2906.0,151.0,NaN


### Float Conversion

Because most numeric columns have blank values on occasion, Pandas saves them as floats (numbers with decimals) by default. Because it doesn't necessarily make sense for them to have decimals (you can't have a fraction of a student, for instance), let's convert all of the floats to `Int64` columns: these are special integer columns that can accomodate blank values. We'll first define a few columns that should stay as floats in the list `floatlist`; feel free to add other columns as needed.


### Dictionary Creation

Next, let's use the dataframe of value labels to create a dictionary, label_dict that will contain each encoded column's names as keys and subdictionaries in each of those as values containing the encoded value and decoded value.

It will look like this:

    {'Bureau of Economic Analysis (BEA) regions (HD2017)': 
      {'0': 'US Service schools',
      '1': 'New England CT ME MA NH RI VT',
      '2': 'Mid East DE DC MD NJ NY PA',
      '3': 'Great Lakes IL IN MI OH WI',
      '4': 'Plains IA KS MN MO NE ND SD',
      '5': 'Southeast AL AR FL GA KY LA MS NC SC TN VA WV',
      '6': 'Southwest AZ NM OK TX',
      '7': 'Rocky Mountains CO ID MT UT WY',
      '8': 'Far West AK CA HI NV OR WA'},
    'Carnegie Classification 2015: Enrollment Profile (HD2017)': 
      {'-2': 'Not applicable, not in Carnegie universe (not accredited or nondegree-granting)',
      '1': 'Exclusively undergraduate two-year',
      '2': 'Exclusively undergraduate four-year',
      '3': 'Very high undergraduate',
      '4': 'High undergraduate',
      '5': 'Majority undergraduate',
      '6': 'Majority graduate',
      '7': 'Exclusively graduate'
      },...
    }

We'll use the label_dict dictionary to convert all encoded values back to their original values. We'll then set the dataframe index to be the institution's name.

### DROP EXTRANEOUS COLUMNS

Finally, we'll drop any empty columns that may have been accidentally created.

In [0]:
# ------------------------------------------------------------------------------
# FLOAT CONVERSION
# ------------------------------------------------------------------------------

float_col = df.select_dtypes(include = ['float64']) # This will select float columns only

floatlist = ['Latitude location of institution (HD2017)','Longitude location of institution (HD2017)']

for col in float_col.columns.values:
    if col not in floatlist:
        df[col] = df[col].astype('Int64')
# You could confirm it worked by uncommenting this code:
# print(df[float_col.columns.values].info())

# ------------------------------------------------------------------------------
# DICTIONARY CREATION AND APPLICATION
# ------------------------------------------------------------------------------

uniquelabels = labels.VariableName.unique().tolist()

label_dict = {}
  
for label in uniquelabels:
    labeldf = labels.loc[labels['VariableName']==label,['Value','ValueLabel']]
    labeldf.rename(columns={'ValueLabel':label}, inplace=True)
    labeldf.set_index('Value',inplace=True)
    df[label] = df[label].apply(lambda x: labeldf.loc[str(x)] if str(x) != '' else '')
    # Dictionary lookup alternative:
    #dfd=labeldf.to_dict(orient='dict')
    #label_dict={**label_dict, **dfd}
    #df[label]=df[label].apply(lambda x: label_dict[label][str(x)] \
    #                          if str(x) != '' else '')

df.set_index('Institution Name', inplace=True)

# ------------------------------------------------------------------------------
# DROP EXTRANEOUS COLUMNS
# ------------------------------------------------------------------------------

# The first list comprehesion identifies both columns that have 'unnamed' in the
# title and that have a total count of NaN values equal to the number of rows 
unnamed_cols = [col for col in df.columns if 'unnamed' in col.lower() and df[col].isna().sum() == df[col].shape[0]]
if len(unnamed_cols) > 0:
  df.drop(columns=unnamed_cols, inplace=True)

Let's have a look at the first few rows of the dataframe to make sure things are looking accurate:

In [34]:
df.head()

,UnitID,FIPS state code (HD2017),Institution (entity) name (HD2017),Institution name alias (HD2017),State abbreviation (HD2017),City location of institution (HD2017),ZIP code (HD2017),Fips County code (HD2017),Longitude location of institution (HD2017),Latitude location of institution (HD2017),Status of institution (HD2017),Sector of institution (HD2017),Level of institution (HD2017),Control of institution (HD2017),Historically Black College or University (HD2017),Degree of urbanization (Urban-centric locale) (HD2017),Carnegie Classification 2015: Undergraduate Instructional Program (HD2017),Carnegie Classification 2015: Graduate Instructional Program (HD2017),Carnegie Classification 2015: Enrollment Profile (HD2017),Carnegie Classification 2015: Undergraduate Profile (HD2017),Carnegie Classification 2015: Size and Setting (HD2017),Land Grant Institution (HD2017),Institution size category (HD2017),Institutional category (HD2017),Bureau of Economic Analysis (BEA) regions (HD2017),Does institution have a tenure system (FLAGS2017),Total price for in-state students living on campus 2017-18 (DRVIC2017),Total price for out-of-state students living on campus 2017-18 (DRVIC2017),Percent of full-time first-time undergraduates awarded Pell grants (SFA1617),Average amount of Pell grant aid awarded to full-time first-time undergraduates (SFA1617),Total enrollment (DRVEF2017)
Institution Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A T Still University of Health Sciences,177834,Missouri,A T Still University of Health Sciences,NaN,Missouri,Kirksville,63501,"Adair County, MO",-92.589183,40.193648,Active - institution active,"Private not-for-profit, 4-year or above",Four or more years,Private not-for-profit,No,Town: Remote,Not Classified (Exclusively Graduate Programs),Research Doctoral: Professional-dominant,Exclusively graduate,Not classified (Exclusively Graduate),Exclusively graduate/professional,Not a Land Grant Institution,"1,000 - 4,999","Degree-granting, graduate with no undergraduat...",Plains IA KS MN MO NE ND SD,Has tenure system,NaN,NaN,NaN,NaN,3723
Aaniiih Nakoda College,180203,Montana,Aaniiih Nakoda College,NaN,Montana,Harlem,59526,"Blaine County, MT",-108.757816,48.484196,Active - institution active,"Public, 2-year",At least 2 but less than 4 years,Public,No,Rural: Remote,"Professions plus arts & sciences, no graduate ...",Not classified (Exclusively Undergraduate),Exclusively undergraduate two-year,"Two-year, higher part-time","Two-year, very small",Land Grant Institution,"Under 1,000","Degree-granting, associate's and certificates ...",Rocky Mountains CO ID MT UT WY,No tenure system,NaN,NaN,85,4091,122
ABC Beauty Academy,459523,Texas,ABC Beauty Academy,NaN,Texas,Richardson,75081,"Dallas County, TX",-96.685333,32.931698,Active - institution active,"Private for-profit, less-than 2-year",Less than 2 years (below associate),Private for-profit,No,City: Midsize,"Not applicable, not in Carnegie universe (not ...","Not applicable, not in Carnegie universe (not ...","Not applicable, not in Carnegie universe (not ...","Not applicable, not in Carnegie universe (not ...","Not applicable, not in Carnegie universe (not ...",Not a Land Grant Institution,"Under 1,000","Nondegree-granting, sub-baccalaureate",Southwest AZ NM OK TX,No tenure system,NaN,NaN,0,NaN,NaN
ABCO Technology,485500,California,ABCO Technology,NaN,California,Inglewood,90304-1551,"Los Angeles County, CA",-118.369774,33.932121,Active - institution active,"Private for-profit, less-than 2-year",Less than 2 years (below associate),Private for-profit,No,Suburb: Large,"Not applicable, not in Carnegie universe (not ...","Not applicable, not in Carnegie universe (not ...","Not applicable, not in Carnegie universe (not ...","Not applicable, not in Carnegie universe (not ...","Not applicable, not in Carnegie universe (not ...",Not a Land Grant Institution,"Under 1,000","Nondegree-granting, sub-baccalaureate",Far West AK CA HI NV OR WA,No tenure system,NaN,NaN,50,4479,50
Abcott Institute,46189

## Export the Decoded Data

If this throws an error message, just re-run the cell until you are prompted to download the file.

In [0]:
today = dt.today().strftime('%Y-%m-%d')
filename = f'{today}-ipeds.xls'
df.to_excel(filename)
files.download(filename)